In [0]:
#importing the tools
from env import env
from src import utils, excel

import openpyxl
import pandas as pd

from pyspark.sql import functions as F
from datetime import datetime
from pyspark.sql.functions import date_format
from openpyxl.styles import NamedStyle

In [0]:
df_raw_sa_csv = spark.read.option("header","true").option("recursiveFileLookup","true").parquet(env["spec_advi_path"])

display(df_raw_sa_csv)
rows = df_raw_sa_csv.count()
print(rows)

In [0]:
#Provider specialist advice activity full dataset from April 2022

df_provider_spec_adv_csv = (df_raw_sa_csv
    .select (
        "EROC_DerMonth",
        "RESP_Org_Code",
        "RESP_Org_Name",
        "Metric_Diverted_Requests",
        "Metric_Processed_Requests",
        "In_scope_of_ERF",
        "EROC_TypeOfSpecialistAdvice_Desc",
        "EROC_Requests",
        "Acute_Status")
    .where (F.col("EROC_DerMonth") >= '2022-04-01')
    .where (F.col("Acute_Status") == 'Acute')
)

df_provider_spec_adv_agg = (
    df_provider_spec_adv_csv.groupBy(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "EROC_TypeOfSpecialistAdvice_Desc",
    "Metric_Diverted_Requests",
    "Metric_Processed_Requests",
    "In_scope_of_ERF")
.   agg(F.sum("EROC_Requests")))

display(df_provider_spec_adv_agg)
rows = df_provider_spec_adv_agg.count() 
print(rows)

# Save as CSV
df_provider_spec_adv_agg_pd = df_provider_spec_adv_agg.toPandas()
df_provider_spec_adv_agg_pd.to_csv('Outputs/SA_MI_Provider.csv', index=False)


# ICB specialist advice full dataset from April 2022

df_ICB_spec_adv_csv = (df_raw_sa_csv
    .select (
        "EROC_DerMonth",
        "EROC_STP_Code",
        "EROC_STP_Name",
        "Metric_Diverted_Requests",
        "Metric_Processed_Requests",
        "In_scope_of_ERF",
        "EROC_TypeOfSpecialistAdvice_Desc",
        "EROC_Requests")
    .where (F.col("EROC_DerMonth") >= '2022-04-01')
)

df_ICB_spec_adv_csv_agg = (df_raw_sa_csv.groupBy(
    "EROC_DerMonth",
    "EROC_STP_Code",
    "EROC_STP_Name",
    "Metric_Diverted_Requests",
    "Metric_Processed_Requests",
    "In_scope_of_ERF",
    "EROC_TypeOfSpecialistAdvice_Desc")
.   agg(F.sum("EROC_Requests"))
    )
display(df_ICB_spec_adv_csv)    

# Save as CSV
df_ICB_spec_adv_csv_pd = df_ICB_spec_adv_csv_agg.toPandas()
df_ICB_spec_adv_csv_pd.to_csv('Outputs/SA_MI_ICB.csv', index=False)

In [0]:
df_provider_spec_adv_csv = (df_raw_sa_csv.filter(F.col("EROC_DerMonth") >= '2022-04-01'))
rows = df_provider_spec_adv_csv.count()
print(rows)

In [0]:
#this is a test